This notebook contains the ML model that predicts log of the sales. It could just as well predict the sales themselves, but given the range of values for sales, it makes more sense to predict the log. An error of \\$50,000 on a 1.5 million dollars game would be less substantial than the same error on a \\$100,000 game.

The notebook assumes that the data set is already cleaned and ready to be used. See the data engineering notebook for the data preparation.

There are three kinds of columns in the data set: numerical (e.g. sales), categorical (e.g. platform, genre), and free-form text (e.g. summary). The numerical data does not require any further preparation. The categorical data will be one-hot encoded. Some categorical columns contain singular values and will be transformed using OneHotEncoder. Other categorical columns contain lists that will use DictVectorizer. Note that the majority of the categorical columns contain numerical values, but these simply represent different classes of whatever data the columns hold. The free-form text will be transformed using TF-IDF vectorizer.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

from tdi_capstone_common_functions import *
# the two functions imported are: standardize_string and pseudo_list_parser

The data, loaded into a pandas data frame, has a lot of columns. While the data engineering notebook dropped columns that were not useful for predictions, specifically a lot of meta-data (e.g., the url for the game on IGDB), there are still columns that may be useful, but may as well not be. They were retained up to this point, but are now dropped. They may be used in future versions of the model. Some, however, can and shouled be dropped regardless, such asname, release_year, closest_match, match_score, id, first_release_date, and release_dates.

In [2]:
# load the data
drop_columns = [
    'name',
    'release_year',
    'closest_match',
    'match_score',
    'id',
    'first_release_date',
    'release_dates',
    'alternative_names',
    'external_games',
    'similar_games',
    'language_supports',
    'status',
    'bundles',
    'collections',
    'parent_game',
    'collection'
    ]

df = (pd.read_csv('data_complete.csv', index_col='index')
      .drop(drop_columns, axis=1))

Since different kinds of columns will be transformed using different transformers, it is easy to create lists of columns according to their kind.

In [3]:
# columns with numerical data
numeric_columns = ['sales_na', 'sales_eu', 'sales_jp', 'sales_other', 'sales_global']

# columns with free-form text
text_columns = ['summary', 'storyline']

# columns that contain lists (imported as strings that look like lists)
list_columns = ['age_ratings', 'game_modes', 'genres', 'themes', 'involved_companies', 'keywords',
               'multiplayer_modes', 'franchises', 'game_engines', 'player_perspectives', 'game_localizations']

# parse the columns that contain pseudo-lists into lists and populate those with a non-list (NaN) with an empty list
df[list_columns] = df[list_columns].applymap(lambda x: pseudo_list_parser(x)).applymap(lambda x: x if isinstance(x, list) else [])

# since most columns won't use OneHotEncoder, it is easier to exclude columns from df.columns than to explicitly list them out
non_ohe_columns = numeric_columns + text_columns + list_columns
# creates a list with the columns that do not appear in any of the above lists
ohe_columns = [column_name for column_name in df.columns if column_name not in non_ohe_columns]

## Building the model

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import DictVectorizer

from sklearn.decomposition import TruncatedSVD # used for dimensionality reduction

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

As mentioned above, there are three kinds of columns: numerical, categorical, and free-form text. Numerical data does not require any transformation at this point. The categorical columns can be divided into two subtypes, columns with singular values that will be transformed using OneHotEncoder and columns with lists of values that will be transformed with DictVectorizer. Lastly, free-form text will use TfidfVectorizer. Each of these kinds of transformations will be stored as a list of transformers with their relevant columns, so they can all eventually be put together into a single list of transformers passed onto the ColumnTransformer of the model.

First to be treated are the OneHotEncoder categorical columns.

In [5]:
ohe_transformers = [('categorical', OneHotEncoder(handle_unknown='ignore'), ohe_columns)]

Then the categorical columns with lists of values. Each one of these lists is first encoded as a dictionary (using a custom class) and then vectorized.

In [6]:
class DictEncoder(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
#         X_trans = pd.DataFrame()
        
#         for column in X.columns:
#             column_trans = [{key: 1 for key in item} for item in X[column]]
#             X_trans[column] = column_trans
        
#         return X_trans
        return [{key: 1 for key in row} for row in X] #, name=X.name)

Because the transformer first needs to encode and then vectorize the list, a pipeline is used.

In [7]:
list_column_vectorizer = Pipeline([
    ('dict_encoder', DictEncoder()),
    ('dict_vectorizer', DictVectorizer())
])

Then a list is created, where every element is the tuple format of a transformer (name, transformer, column(s)) for every column. This way, it is possible to access specific transformers by its name via the named_steps method.

In [8]:
list_column_transformers = [(f'list_vect_{column}', list_column_vectorizer, f'{column}') for column in list_columns]

Lastly, the free-form text columns are treated, where the strings are first standardized and then vectorized using TF-IDF. Very rare words are excluded, represented by the min_df value. Additionally, a relatively low max_df is set to exclude words that appear in a lot of game descriptions, since we want to use the most impactful words.

In [9]:
def text_column_preprocessing(series):
    """
    Standardizes a series containing free-form strings.
    
    Parameter
    ---------
    series : pandas.Series()
        A series of strings to be standardized, retaining only alphanumeric characters,
        changing East Asian characters into Latin ones, and removing symbols and parentheses.
    
    Returns
    -------
    pandas.Series
        A series of standardized strings.
    """
    
    return series.map(standardize_string)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from spacy.lang.en.stop_words import STOP_WORDS

STOP_WORDS = STOP_WORDS.difference({'he','his','her','hers'}).union({'ll', 've'})

text_column_vectorizer = Pipeline([
    ('standardize_text', FunctionTransformer(text_column_preprocessing)),
    ('tfidf', TfidfVectorizer(min_df=20, max_df=0.5, stop_words=list(STOP_WORDS)))
])

Much like in the case with the list columns, a list of tuples is created, each referring to an individual free-fore text column.

In [11]:
text_column_transformers = [(f'text_{column}', text_column_vectorizer, f'{column}') for column in text_columns]

It is now possible to build the model. All the features will undergo their respective transformations (remember that the numerical columns do not need any at this point).

In [12]:
features = ColumnTransformer(
    transformers = list_column_transformers + ohe_transformers + text_column_transformers,
    remainder='drop')

The model was trained with several regressors alongside appropriate parameter grids. These include LinearRegression, Ridge, RandomForestRegressor, and KNeighborsRegressor. Out of all of them, Ridge seemed to performed the best. In order to streamline building different versions of the model, a function allows for a quick customization of the regressor.

Note that the data set ends up having quite a lot of features, particularly due to the free-form text vectorizer. Given the number of observations, this needs to be taken into account. To balance that, some kind of dimensionality reduction can help to improve the model. Post-transformation, the data is stored in a sparse matrix, and since PCA doesn't work with sparse data, we'll use TruncatedSVD.

In [13]:
def generate_regressor(requested_model='ridge', hyperparameters=None, requested_dr='default', cv=5, n_jobs=2):
    """
    Creates a regressor with grid search cross-validation.
    
    Parameters
    ----------
    requested_model : string or sklearn model, default 'ridge'
        The name of the model as a string or an instantiated model
        object itself. Currently supported models: LinearRegression, Ridge,
        RandomForestRegressor, KNeighborsRegressor. Defaults to Ridge
        if anything else is given.
    hyperparameters : dict, default None
        The hyperparameters for grid search cross-validation based on
        the regressor. If not passed, defaults to the built-in ones.
    requested_dr : string or list, default 'default'
        Indicates how dimensionality reduction using TruncatedSVD should
        work. Accepted strings are 'default' for the default values based
        on the model chosen, 'aggressive' to reduce to hundreds of features,
        or 'relaxed' to reduce to tens of features. Alternatively, a list
        can be passed with the possible values.
    cv : int, default 5
        The number of folds in the cross validation.
    n_jobs : int, default 2
        The number of parallel jobs for the cross validation.
    
    Returns
    -------
    GridSearchCV object
        An object based on the requested model or the default if that
        type of regressor is not supported by the function. The parameter
        grid is also set for the specified model.
    
    Notes
    -----
    The currently supported models are linear regression, ridge, random
    forest, and k-nearest neighbors. All but the last one use a relaxed
    dimensionality reduction (n_components = [100, 250, 500]) by default,
    with k-nearest neighbors using an aggressive one
    (n_components = [10, 20, 30]). The built-in default hyperparameters
    for each are as follows:
        Ridge :
            'alpha': [0.1, 1.0, 10.0]
        Random Forest :
            'n_estimators': [10, 50, 100, 200, 300],
            'max_depth': [None, 10, 20, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        K-Nearest Neighbors :
            'n_neighbors': [3, 5, 8, 10, 15]  
    """

    #
    if isinstance(requested_model, str):
        supported_models = {
            'linearregression': LinearRegression(),
            'lr': LinearRegression(),
            'linear': LinearRegression(),
            'ridge': Ridge(),
            'randomforestregressor': RandomForestRegressor(),
            'randomforest': RandomForestRegressor(),
            'kneighborsregressor': KNeighborsRegressor(),
            'knn': KNeighborsRegressor()
        }
        
        model = supported_models.get(requested_model.lower(), Ridge())
        
    # The default values for dimensionality reduction
    dr_values = {
        'relaxed': [100, 250, 500],
        'aggressive': [10, 20, 30] # for, e.g., KNN
    }
    
    # if the function was passed a specific list of dimensionality reduction values, it will use that list
    if isinstance(requested_dr, list):
        dr = requested_dr
    else:
        # if a string was passed, it will use one of the two built-in options, either aggressive or relaxed dimensionality
        # reduction; or if the string was 'default' or another non-supported string, it will use a default set of values.
        # These are based on the model requested, and so a function is used to determine which of the two built-in options
        # to use.
        def find_default_dr(model_for_dr):
            aggressive_dr_models = [KNeighborsRegressor] # list of models which default to aggressive dimensionality reduction
            
            if model_for_dr in aggressive_dr_models:
                return dr_values['aggressive']
            
            return dr_values['relaxed']
        
        # find the correct built-in value for dimensionality reduction
        dr = dr_values.get(requested_dr, find_default_dr(model))

    # Initiliazes the dimensionality reduction object
    svd = TruncatedSVD()
    
    # Initializes the parameter grid with the dimensionality reduction values
    param_grid = {'dim_reduction__n_components': dr}
    
    # If no parameter grid was passed, use the built-in values
    if not hyperparameters:
        
        if isinstance(model, Ridge):
            hyperparameters = {
                'alpha': [0.1, 1.0, 10.0]
            }
        elif isinstance(model, RandomForestRegressor):
            hyperparameters = {
                'n_estimators': [10, 50, 100, 200, 300],
                'max_depth': [None, 10, 20, 50],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }
        elif isinstance(model, KNeighborsRegressor):
            hyperparameters = {
                'n_neighbors': [3, 5, 8, 10, 15]   
            }
        else: # default if model does not have hyperparameters, e.g. linear regression
            hyperparameters = {}
    
    # renames the keys so they can be properly accessed
    hyperparameters = {f'regressor__{k}' : v for k, v in hyperparameters.items()}
        
    # Updates the parameter grid (currently only with dimensionality reduction) with the regressor's hyperparameters.
    param_grid.update(hyperparameters)
        
    estimator = Pipeline([
        ('dim_reduction', svd),
        ('regressor', model)
    ])

    gs = GridSearchCV(
        estimator,
        param_grid=param_grid,
        cv=cv,
        n_jobs=n_jobs
    )

    return gs

It is then possible to build the model with different kinds of regression algorithms or parameter grids. 

In [72]:
model = Pipeline([
    ('features', features),
    ('main_regressor', generate_regressor('ridge', cv=10))
])

Alternatively, we can generate several models, each with a different regressor

In [59]:
# algorithms = ['lr', 'ridge', 'randomforest', 'knn']

# models = dict()

# for algorithm in algorithms:
#     models[algorithm] = Pipeline([
#         ('features', features),
#         ('main_regressor', generate_regressor(algorithm, cv=10, n_jobs=-1))
#     ])

## Training the model

As mentioned above, the numerical columns are only the sales, which are also the (basis of the) labels, so these can be dropped from the feature matrix passed onto the model. The labels are going to be the log of the global sales. In the future, it would be possible to have the model predict localized sales (e.g. EU, US, etc).

In [73]:
X = df.drop(numeric_columns, axis=1)
y = np.log(df['sales_global'])

The data is then split into a training set and a test set.

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

The model is then fit with the training set.

In [75]:
model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('list_vect_age_ratings',
                                                  Pipeline(steps=[('dict_encoder',
                                                                   DictEncoder()),
                                                                  ('dict_vectorizer',
                                                                   DictVectorizer())]),
                                                  'age_ratings'),
                                                 ('list_vect_game_modes',
                                                  Pipeline(steps=[('dict_encoder',
                                                                   DictEncoder()),
                                                                  ('dict_vectorizer',
                                                                   DictVectorizer())]),
                                                  'game_modes'),
                                                 ('list_vect_genres',
                                                  Pipeline(steps=[('di...
                                                                                               'themselves',
                                                                                               'many',
                                                                                               'yourself',
                                                                                               'though',
                                                                                               'among',
                                                                                               'whereas',
                                                                                               'any',
                                                                                               'nine',
                                                                                               'front',
                                                                                               'whether',
                                                                                               '‘re',
                                                                                               'whoever', ...]))]),
                                                  'storyline')])),
                ('main_regressor',
                 GridSearchCV(cv=10,
                              estimator=Pipeline(steps=[('dim_reduction',
                                                         TruncatedSVD()),
                                                        ('regressor',
                                                         Ridge())]),
                              n_jobs=2,
                              param_grid={'dim_reduction__n_components': [100,
                                                                          250,
                                                                          500],
                                          'regressor__alpha': [0.1, 1.0,
                                                               10.0]}))])

In [71]:
# for model in models.values():
#     model.fit(X_train, y_train)


KeyboardInterrupt



The model can then be serialized and saved into a file so it doesn't have to be trained again. A container (list) object is created that will hold the model itself, as well as the train/test split of the data in order to maintain consistency.

In [91]:
import pickle

data_container = [model, X_train, X_test, y_train, y_test]

In [92]:
with open('ridge.pickle', 'wb') as f:
    pickle.dump(data_container, f)

If the model has already been pickled, it can be imported.

In [93]:
with open('ridge.pickle', 'rb') as f:
    unpickled_container = pickle.load(f)

model, X_train, X_test, y_train, y_test = data_container

Predictions can then be made:

In [95]:
y_pred = model.predict(X_test)

It's then possible to observe some metrics on the performance of the model.

In [96]:
from sklearn import metrics

In [97]:
print(f"Mean absolute error: {metrics.mean_absolute_error(y_test, y_pred)}")
print(f"Mean squared error: {metrics.mean_squared_error(y_test, y_pred)}")
print(f"R^2: {metrics.r2_score(y_test, y_pred)}")

Mean absolute error: 0.7917553133639861
Mean squared error: 0.9828128160787609
R^2: 0.579290374875455


In [ ]:
#y_pred = pipe.predict(X_test)

In [ ]:
# print(f"Mean absolute error: {metrics.mean_absolute_error(y_test, y_pred)}")
# print(f"Mean squared error: {metrics.mean_squared_error(y_test, y_pred)}")
# print(f"R^2: {metrics.r2_score(y_test, y_pred)}")

In [ ]:
# pipe.score(X_test, y_test)

## Data visualization

In [ ]:
# import pandas as pd
# import pickle
from ipywidgets import widgets
import matplotlib.pyplot as plt

# df_housing = pd.read_csv('small_data/Housing_Train.csv')
# with open('small_data/Housing_Train_dict.pkl', "rb") as f:
#     features_dict = pickle.load(f)

# X = df_housing.drop('SalePrice', axis=1)
# y = df_housing['SalePrice']

def housing_plot(X, y):
    def plotter(column):
        valid_rows = X[column].notna()
        plt.plot(X.loc[valid_rows, column], y[valid_rows], '.', color='k')
#         plt.xlabel(features_dict[column])
        plt.ylabel('Global sales (mil $)')
    
    return plotter

# dropdown_values = {f"{k}: {features_dict[k]}": k for k in sorted(X.columns)}
# widgets.interact(housing_plot(X, y), column=dropdown_values);
widgets.interact(housing_plot(X_train, y_train), column=X_train.columns);

# dropdown_values = {f"{k}: {features_dict[k]}": k for k in sorted(X.columns)}
# widgets.interact(housing_plot(X, y), column=dropdown_values);

# NOTE: This doesn't currently work with the columns that have lists as values

In [ ]:
# scatter plot of actual versus predicted values

In [ ]:
# def plot_predicted_vs_actual(actual_values, predicted_values):
#     plt.figure(figsize=(10, 6))
#     plt.scatter(actual_values, predicted_values, alpha=0.5, label='Predicted vs Actual')
#     plt.plot([min(actual_values), max(actual_values)], [min(actual_values), max(actual_values)], color='red', linestyle='--', label='Perfect Prediction')
#     plt.xlabel('Actual Log Sales')
#     plt.ylabel('Predicted Log Sales')
#     plt.title('Actual vs Predicted Log Sales')
#     plt.legend()
#     plt.show()

# plot_predicted_vs_actual(y_test, y_pred)
# plot_predicted_vs_actual(np.exp(y_test), np.exp(y_pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_predicted_vs_actual_enhanced(actual_values, predicted_values, extras='log of sales'):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=actual_values, y=predicted_values, alpha=0.5)
    sns.lineplot(x=[min(actual_values), max(actual_values)], y=[min(actual_values), max(actual_values)], color='red', linestyle='--')
    plt.xlabel(f'Actual values ({extras})')
    plt.ylabel(f'Predicted values ({extras})')
    plt.title(f'Actual vs predicted values ({extras})')
    plt.show()

plot_predicted_vs_actual_enhanced(y_test, y_pred)
plot_predicted_vs_actual_enhanced(np.exp(y_test), np.exp(y_pred), 'mil $')

In [ ]:
np.random.seed(0)
actual_values = np.random.normal(loc=10, scale=2, size=100)
predicted_values = actual_values + np.random.normal(loc=0, scale=1, size=100)

# plot_predicted_vs_actual(actual_values, predicted_values)
plot_predicted_vs_actual_enhanced(actual_values, predicted_values, 'mil $')

In [ ]:
# svd = TruncatedSVD()

# regressor = Ridge()
# #regressor = RandomForestRegressor()
# # regressor = KNeighborsRegressor()

# param_grid = {
# # relaxed dimensionality reduction:
#     'dim_reduction__n_components': [100, 250, 500],
# # aggresive dimensionality reduction (for, e.g., KNN):
# #     'dim_reduction__n_components': [10, 20, 30],

# # Ridge hyperparameters
#    'regressor__alpha': [0.1, 1.0, 10.0]
# # KNN hyperparameters
# #     'regressor__n_neighbors': [3, 5, 8, 10, 15]
# # RandomForest hyperparameters:
# #     'regressor__n_estimators': [10, 50, 100],#, 200, 300],
# #     'regressor__max_depth': [None, 10, 30], # 20, 50],
# #     'regressor__min_samples_split': [2, 5, 10],
# #     'regressor__min_samples_leaf': [1, 2, 4]
# }

# estimator = Pipeline([
#     ('dim_reduction', svd),
#     ('regressor', regressor)
# ])

# gs = GridSearchCV(
#     estimator,
#     param_grid=param_grid,
#     cv=5,
#     n_jobs=2
# )

# pipe = Pipeline([
#     ('features', features),
#     ('main_regressor', gs)
# ])

# pipe.fit(X_train, y_train)